In [2]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [3]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [4]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [4]:
usecase_name = "SignUp"
project_name = "RSL"
main_flow_prompt="""
Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.
"""
prompt_all = """
Use Case: Signup

As an Anonymous user, I want to sign up to become a user.

Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.

Exception flow: An email already exists in the system
At step 3 of the main flow: The system validates if the submitted email does not exist.
Step 4: The system displays a message that the submitted email does not exist.

Exception flow: Signup with different passwords
At step 6 of the main flow: The system validates the passwords are different.
Step 7: The system displays a message that the passwords are different.

Precondition: The user's email cannot already be registered in the system.
Postcondition: The new user is created in the system.
"""
exc_prompt = [
"""Exception flow: An email already exists in the system
At step 3 of the main flow: The system validates if the submitted email does not exist.
Step 4: The system displays a message that the submitted email does not exist.""", 
"""Exception flow: Signup with different passwords
At step 6 of the main flow: The system validates the passwords are different.
Step 7: The system displays a message that the passwords are different."""]

In [5]:
for i in range(1,2):
    print(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.20\\old filter\\{project_name}\\{usecase_name}-{i}.txt")

D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\RSL\SignUp-1.txt


In [6]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(exc)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")
    filtercontent+=  "\n" + sub_gpt_response
    sub_gpt_response = ask(promptSubScenario, client, model)


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.23\\run propose\\{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Email': {'condition': {'valid': 'the submitted email does not exist in the system', 'invalid': 'the submitted email already exists in the system'}, 'type': 'text field'}, 'Confirm Password': {'condition': {'valid': 'the passwords match', 'invalid': 'the passwords are different'}, 'type': 'text field'}}
1. Signup with an email that already exists in the system
2. Signup with mismatched password and confirm password fields

Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.


main_gpt_response:1. Successful Signup
2. Password Mismatch
3. Invalid Email Format
4. Email Already Exists
5. Weak Password

Exception fl

In [5]:
usecase_name = "SignUp"
project_name = "RSL"
main_flow_prompt="""
Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.
"""
prompt_all = """
Use Case: Signup

As an Anonymous user, I want to sign up to become a user.

Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.

Exception flow: An email already exists in the system
At step 3 of the main flow: The system validates if the submitted email does not exist.
Step 4: The system displays a message that the submitted email does not exist with the content "The submitted email does not exists".
Step 5: The user fills in an exists email.
Go back to step 4 in the basic flow and continue with the steps from step 4

Exception flow: Signup with different passwords
At step 6 of the main flow: The system validates the passwords are different.
Step 7: The system displays a message that the passwords are different with the content "The passwords are different".
Go back to step 5 in the basic flow and continue with the steps from step 5

Precondition: The user's email cannot already be registered in the system.
Postcondition: The new user is created in the system.
"""
exc_prompt = [
"""Exception flow: An email already exists in the system
At step 3 of the main flow: The system validates if the submitted email does not exist.
Step 4: The system displays a message that the submitted email does not exist.
Step 5: The user fills in an exists email.
Go back to step 4 in the basic flow and continue with the steps from step 4""", 
"""Exception flow: Signup with different passwords
At step 6 of the main flow: The system validates the passwords are different.
Step 7: The system displays a message that the passwords are different.
Go back to step 5 in the basic flow and continue with the steps from step 5"""]

In [6]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(exc)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")
    filtercontent+=  "\n" + sub_gpt_response
    sub_gpt_response = ask(promptSubScenario, client, model)


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.23\\run propose\\{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Email': {'condition': {'valid': 'must be entered, must not already exist in the system', 'invalid': 'already exists in the system'}, 'type': 'text field'}, 'Password': {'condition': {'valid': 'must be entered', 'invalid': ''}, 'type': 'text field'}, 'Confirm Password': {'condition': {'valid': 'must match Password', 'invalid': 'does not match Password'}, 'type': 'text field'}}
1. Signup with email that already exists in the system
2. Signup with password and confirm password that do not match

Main Scenario:
Step 1: The system opens the signup page in the browser.
Step 2: The user fills in the name.
Step 3: The user fills in the email.
Step 4: The user fills in the password.
Step 5: The user fills in the confirm password.
Step 6: The user selects the action signup.
Step 7: The system validates the passwords are correct.
Step 8: The system displays a message that the user was successfully created in the system.


main_gpt_response:1. Successful Signup
2. Password Mismatch
3. Inv